Assignment 3 notes 

In [1]:
#import packages 
import os

import pandas as pd 
import gensim.downloader as api
from collections import Counter
import string
#packages for install
#pandas gensim

Flow of assignment (in one script):

- Loads the song lyric data
- Downloads/loads a word embedding model via ```gensim``` (see below)

- Takes a given word as an input and finds the most similar words via word embeddings
- Find how many songs for a given artist feature terms from the expanded query
- Calculate the percentage of that artist's songs featuring those terms
- Print and/or save results in an easy-to-understand way
    - For example, "45% of {ARTIST}'s songs contain words related to {SEARCH TERM}"

In [17]:
#load songs
filepath = os.path.join("in", "Spotify Million Song Dataset_exported.csv")

songs = pd.read_csv(filepath)

In [18]:
#load model
model = api.load("glove-wiki-gigaword-50")

In [6]:
# Function to check if artists exists in corpus, makes both input and corpus lower case 
def artist_find(artist_q):
    """
    The function takes a string, if the string matches with an artist name, 
    a subset of the songs dataframe is returned, only with the given artist's songs.
    """
    artist_q = artist_q.lower()
    songs["artist"] = (songs["artist"]).str.lower()
    
    if (songs['artist'].eq(artist_q).any()):
        artist_cor = songs[songs["artist"] == artist_q]
        artist_songs = list(artist_cor["text"])
        
    else: print('Artist not found') 

    return artist_songs

In [7]:
ts_songs = artist_find("Taylor Swift")

In [15]:
def query_expansion(keyword):
    query = model.most_similar(keyword)
    query_ls = ([i[0] for i in query])
    query_ls.append(keyword)

    return query_ls

In [20]:
query_expansion("breakup")

['break-up',
 'disintegration',
 'protracted',
 'bankruptcy',
 'divorce',
 'dispute',
 'demise',
 'ending',
 'split',
 'wars',
 'breakup']

In [13]:
def clean_tokens(text):
    tokens = text.lower().split()
    tokens_cleaned = []
    
    for token in tokens: 
        strip_punct = token.strip(string.punctuation)
        if not strip_punct == "":
            tokens_cleaned.append(strip_punct)
    
    return tokens_cleaned

In [15]:
tokens_cleaned = clean_tokens(ts_songs[1])

In [16]:
def keyword_count(keyword, song_texts, artist_q):

    query_c = Counter(query_expansion(keyword))
    song_count = 0

    for text in range(len(song_texts)):
        tokens_cleaned = clean_tokens(song_texts[text])
                        
        tokens_c = Counter(tokens_cleaned)
        keyword_count = (tokens_c & query_c).total()
        if keyword_count > 0: 
            song_count +=1

    songs_perc = round(((song_count/len(ts_songs))*100), 2)

    return f"{songs_perc}% of {artist_q}'s songs contains words related to {keyword}"    

In [19]:
keyword_count("mad", ts_songs, "Taylor Swift")

"9.88% of Taylor Swift's songs contains words related to mad"

In [67]:
def save_output(keyword):
    keywords_output = open("out/keywords.txt", "r")
    text = f"\n{keyword}: " + str(query_expansion(keyword)[0:10]) 

    if text in (keywords_output.readlines()): 
        print(text)
        keywords_output.close()
    else: 
        keywords_output.close()
        keywords_output = open("out/keywords.txt", "a")
        keywords_output.write(text)
        keywords_output.close()

In [70]:
save_output("hello")